这里提供 Tensorflow 完成作业的框架代码。作业在最后部分已给出

## 运行环境

In [2]:
%load_ext watermark
%watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
2017-03-21T15:09:15

CPython 2.7.6
IPython 5.3.0

compiler   : GCC 4.8.4
system     : Linux
release    : 4.9.12-moby
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit


In [3]:
import tensorflow as tf
print(tf.__version__)

1.0.1


In [4]:
import tensorflow as tf
import numpy as np

In [5]:
def generate_weight_and_bias(input_dim):
    # 1 x input_dim 行向量，相当于课程中的 w^T
    w = tf.Variable(tf.random_uniform([1, input_dim], -1, 1))
    # b, 1 * 1
    bias = tf.Variable(tf.zeros([1, 1]))
    return (w, bias)

**此处生成一些随机数据，只是为演示所用。最后一个作业的数据请参考 ch3/code/README.md**

这里随机生成两类容易分隔的数据

In [6]:
X = np.concatenate((np.random.rand(2, 50), np.random.rand(2, 50) + 1), axis=1)
# 注意这里要转换成 float32，否则默认可能是精度更高的 float64，和后面 W, b 的类型对应，Tensorflow 的矩阵乘法不会自己做类型转化。
# 也可以在矩阵乘法中使用 tf.cast(x, tf.float32) 来转换类型
X = X.astype(np.float32)
y = np.asarray([0] * 50 + [1] * 50)

## 逻辑回归 (Sigmoid + Cross entropy)

In [7]:
W, b = generate_weight_and_bias(2)
nn_output = tf.sigmoid(tf.add(tf.matmul(W, X), b))

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    res = sess.run(nn_output)

In [11]:
res.shape

(1, 100)

### 利用 numpy 中的操作理解矩阵乘法

因为我们的输入 X 是 100 个样本，每一列是一个样本（样本表示为列向量，和课程中的表示一致）。这里看到网络随机初始化之后，输出的结果是一个 (1, 100) 的结果。

因为 Tensorflow 中矩阵操作和 numpy 很类似，如果一时不容易理解，可以先用 numpy 的操作来辅助理解

In [9]:
W_np = np.random.uniform(-1, 1, (1, 2))
b_np = np.zeros((1, 1))

In [10]:
W_np # 1 * 2 矩阵

array([[-0.29825738, -0.28873677]])

In [11]:
b_np # 1 * 1 矩阵（列向量）

array([[ 0.]])

In [12]:
X[np.newaxis, 0].shape

(1, 100)

In [13]:
# numpy 中矩阵乘法调用的是 dot，而 tensorflow 中调用的是 tf.matmul
raw_output = W_np.dot(X) + b_np

In [14]:
raw_output.shape

(1, 100)

In [15]:
# 我们可以只取样本的一列（一个样本），来更好地观察
raw_output = W_np.dot(X[:, 0] + b_np)

## 上面这个语句会出错，因为 X[0] 在 numpy 里得到的是一个行向量

ValueError: shapes (1,2) and (1,2) not aligned: 2 (dim 1) != 1 (dim 0)

In [16]:
# 我们需要把输入表示成列向量，比较
print(X[:, 0])
# 以下两种方法都可以变成列向量
print(X[:, [0]])
print(X[:, 0].reshape(2, 1))

[ 0.63340271  0.27566573]
[[ 0.63340271]
 [ 0.27566573]]
[[ 0.63340271]
 [ 0.27566573]]


In [17]:
raw_output = W_np.dot(X[:, [0]] + b_np)

In [18]:
print(raw_output)

[[-0.26851186]]


而之前得到的 (1, 100) 结果不过是把 100 个样本按列进行拼接，一次性得到 100 个样本的结果而已。

### Tensorflow 中的 sigmoid + cross entropy

如果看过作业中 "Neural Network and Deep Learning" Simgoid + cross entropy 相关章节的话，应该能理解，sigmoid + cross entropy 在求导数的时候，可以把分母进行约简，少做一些计算。

Tensorflow 利用了这一特性进行计算约简，因此其有一个专门的 cost 层，叫 `tf.nn.sigmoid_cross_entropy_with_logits`

所谓的 logit，其实是 sigmoid 的反函数，也就是说这个 cost 函数，需要的输入是 sigmoid 变换之后的结果，再取其反函数 logit。

这么这个结果，其实就是我们神经网络输出节点的原始结果，即在 sigmoid 变换之前的结果。


为了配合这个 cost 层进行计算约简，我们需要先准备一个 sigmoid 激活之前的结果，在训练时使用激活之前的结果即可，在『预测』时才使用 sigmoid 激活之后的结果

In [19]:
tf.reset_default_graph()

In [20]:
W, b = generate_weight_and_bias(2)
# X, y 这里是用 numpy array 的方式， Tensorflow 会将他们变成 Constant tensor 放入 graph 当中
# **请尝试将 X, y 改成 tensorflow 的 placeholder 形式**
nn_raw_output = tf.add(tf.matmul(W, X), b)
final_output = tf.sigmoid(nn_raw_output)

In [21]:
cost = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=nn_raw_output, labels=y.reshape(1, 100).astype(np.float32))
)

In [22]:
def accuracy(preds, labels):
    return ((preds > 0.5) == labels).sum() / float(len(labels))

In [23]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cost)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        sess.run(train_step)
        if i % 50== 0:
            preds = sess.run(final_output)
            print(accuracy(preds, y))

0.41
0.74
0.87
0.95
0.96
0.97
0.98
0.99
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


**请通过改动框架代码，来实现作业剩余的部分**
- X, y 这里是用 numpy array 的方式， Tensorflow 会将他们变成 Constant tensor 放入 graph 当中。**请尝试将 X, y 改成 tensorflow 的 placeholder 形式**
- 实现随机梯度下降的线性回归
- 针对给定二分类数据，实现无隐层神经网络，单隐层神经网络。比较准确率并绘制分界面

